In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle("df_cleaned.pkl")

In [3]:
df.head()

,Article,Title,Paragraphs,References_internal_clean,Paragraphs_cleaned
1,Article 1,Scope,This Regulation lays down uniform rules concer...,[460],This Regulation lays down uniform rules concer...
2,Article 2,Supervisory powers,For the purposes of ensuring compliance with t...,[],For the purposes of ensuring compliance with t...
3,Article 3,Application of stricter requirements by instit...,This Regulation shall not prevent institutions...,[],This Regulation shall not prevent institutions...
4,Article 4,Definitions,"1. For the purposes of this Regulation, the ...","[4, 2, 115, 25, 71, 301, 113, 1]","1. For the purposes of this Regulation, the ..."
5,Article 5,Definitions specific to capital requirements f...,"For the purposes of Part Three, Title II, the ...",[],"For the purposes of Part Three, Title II, the ..."


In [4]:
del df['Paragraphs']

In [5]:
df.head()

,Article,Title,References_internal_clean,Paragraphs_cleaned
1,Article 1,Scope,[460],This Regulation lays down uniform rules concer...
2,Article 2,Supervisory powers,[],For the purposes of ensuring compliance with t...
3,Article 3,Application of stricter requirements by instit...,[],This Regulation shall not prevent institutions...
4,Article 4,Definitions,"[4, 2, 115, 25, 71, 301, 113, 1]","1. For the purposes of this Regulation, the ..."
5,Article 5,Definitions specific to capital requirements f...,[],"For the purposes of Part Three, Title II, the ..."


In [6]:
def strtonumber(numbers):
    numbers = list(map(int, numbers))
    return numbers

In [7]:
df['References'] = df['References_internal_clean'].apply(strtonumber)

In [8]:
df.head()

,Article,Title,References_internal_clean,Paragraphs_cleaned,References
1,Article 1,Scope,[460],This Regulation lays down uniform rules concer...,[460]
2,Article 2,Supervisory powers,[],For the purposes of ensuring compliance with t...,[]
3,Article 3,Application of stricter requirements by instit...,[],This Regulation shall not prevent institutions...,[]
4,Article 4,Definitions,"[4, 2, 115, 25, 71, 301, 113, 1]","1. For the purposes of this Regulation, the ...","[4, 2, 115, 25, 71, 301, 113, 1]"
5,Article 5,Definitions specific to capital requirements f...,[],"For the purposes of Part Three, Title II, the ...",[]


In [9]:
frame = pd.DataFrame(columns = ['Article1', 'Article1_Paragraphs', 'Article2', 'Article2_Paragraphs'])
frame.index = frame.index + 1

In [10]:
articles1 = list()
paragraphs1 = list()
articles2 = list()
paragraphs2 = list()
label = list()
articles1b = list()
paragraphs1b = list()
articles2b = list()
paragraphs2b = list()
labelb = list()
for x in range (1, len(df)):
    numbers = df['References_internal_clean'][x]
    numbers = list(map(int, numbers))
    for y in range (1, len(df)):
        if y in numbers:
            articles1.append(df['Article'][x])
            paragraphs1.append(df['Paragraphs_cleaned'][x])
            articles2.append(df['Article'][y])
            paragraphs2.append(df['Paragraphs_cleaned'][y])
            label.append(1)
        else:
            articles1b.append(df['Article'][x])
            paragraphs1b.append(df['Paragraphs_cleaned'][x])
            articles2b.append(df['Article'][y])
            paragraphs2b.append(df['Paragraphs_cleaned'][y])
            labelb.append(0)        

In [11]:
crr_articles = { 'articles1': articles1,'paragraphs1': paragraphs1, 'articles2': articles2, 'paragraphs2': paragraphs2, 'label': label}

frame = pd.DataFrame(crr_articles, columns = ['articles1', 'paragraphs1', 'articles2', 'paragraphs2', 'label'])

In [12]:
crr_articles2 = { 'articles1': articles1b,'paragraphs1': paragraphs1b, 'articles2': articles2b, 'paragraphs2': paragraphs2b, 'label': labelb}

frame2 = pd.DataFrame(crr_articles2, columns = ['articles1', 'paragraphs1', 'articles2', 'paragraphs2', 'label'])

In [13]:
len(frame2)

272339

In [14]:
len(frame)

1190

In [15]:
frame2_sample = frame2.sample(n=1190)

In [16]:
len(frame2_sample)

1190

In [17]:
frames = [frame, frame2_sample]

In [18]:
result = pd.concat(frames)

In [19]:
len(result)

2380

In [110]:
import Levenshtein
Levenshtein.ratio('hello world', 'hello')


ModuleNotFoundError: No module named 'Levenshtein'

In [113]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

SyntaxError: invalid character in identifier (<ipython-input-113-a03f1068e8e9>, line 2)

In [20]:
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

def minkowski(content_a, content_b):
    vector1 = text_to_vector(content_a)
    vector2 = text_to_vector(content_b)

    minkowski_result = minkowski_distance(vector1, vector2, 2)
    return minkowski_result

In [116]:
def manhattan_distance(x,y):
    x = text_to_vector(x)
    y = text_to_vector(y)
    dist = sum(abs(a-b) for a,b in zip(x,y))
    return dist

In [117]:
import scipy.spatial.distance as dist
vector1 = text_to_vector('hey what is up')
vector2 = text_to_vector('no sorry hey')
dist.euclidean(vector1, vector2)

TypeError: unsupported operand type(s) for *: 'Counter' and 'Counter'

In [118]:
import gensim

C:\Users\skamuf\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [122]:
#load word2vec model, here GoogleNews is used
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
#two sample sentences 
s1 = 'the first sentence'
s2 = 'the second text'

#calculate distance between two sentences using WMD algorithm
distance = model.wmdistance(s1, s2)

print ('distance = %.3f' % distance)

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

In [21]:
import nltk
nltk.edit_distance("humpty yup hey yeah", "dumpty nope yeah")

6

In [22]:
result.head()

,articles1,paragraphs1,articles2,paragraphs2,label
0,Article 1,This Regulation lays down uniform rules concer...,Article 460,1. The Commission shall be empowered to adop...,1
1,Article 4,"1. For the purposes of this Regulation, the ...",Article 1,This Regulation lays down uniform rules concer...,1
2,Article 4,"1. For the purposes of this Regulation, the ...",Article 2,For the purposes of ensuring compliance with t...,1
3,Article 4,"1. For the purposes of this Regulation, the ...",Article 4,"1. For the purposes of this Regulation, the ...",1
4,Article 4,"1. For the purposes of this Regulation, the ...",Article 25,The Tier 1 capital of an institution consists ...,1


In [23]:
result['cosine_similarity'] = result.apply(lambda x: get_result(x['paragraphs1'], x['paragraphs2']), axis=1)

In [ ]:
result['edit_distance'] = result.apply(lambda x: nltk.edit_distance(x['paragraphs1'], x['paragraphs2']), axis=1)

In [ ]:
result.head()

In [80]:
from sklearn.model_selection import train_test_split 
 
X_train, X_test, y_train, y_test = train_test_split(result['cosine_similarity'], result['label'], test_size=0.3) 


In [81]:
from sklearn.ensemble import RandomForestClassifier 

rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456) 
rf.fit(X_train, y_train) 


ValueError: Expected 2D array, got 1D array instead:
array=[0.7687724  0.5233333  0.34099716 ... 0.7725663  0.19415483 0.64828753].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.